In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

project_selector = widgets.ProjectSelectionWidget()
project_selector.load_dropdown_box()

measurements_selector = widgets.MeasurementMultipleSelectionWidget()

simulations_selector = widgets.SimulationMultipleSelectionWidget()

codes_selector = widgets.CodeMultipleSelectionWidget()
codes_selector.load_selector()

analysis_widgets = widgets.ObjectPropertiesWidgets("Analysis")

analysis_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def load_project_objects_metadata(change):
    if project_selector.dropdown.value != -1:
        simulations_selector.load_selector(project_selector.dropdown.value)
        measurements_selector.load_selector(project_selector.dropdown.value)

def create_analysis_openbis(b):
    analysis_props = {}
    for prop in CONFIG["objects"]["Analysis"]["properties"]:
        analysis_props[prop] = analysis_widgets.properties_widgets[prop].value
        
    project_permid = project_selector.dropdown.value
    project_results_collection = OPENBIS_SESSION.get_collections(project = project_permid, code = "ANALYSIS_COLLECTION")
    analysis_parents = []
    analysis_parents.extend(list(measurements_selector.selector.value))
    analysis_parents.extend(list(simulations_selector.selector.value))
    analysis_parents.extend(list(codes_selector.selector.value))
    
    if len(project_results_collection) == 0:
        analysis_collection = utils.create_openbis_collection(
            OPENBIS_SESSION, 
            project = project_selector.dropdown.value, 
            code = "ANALISYS_COLLECTION", type = "COLLECTION", props = {"$name": "Analysis"}
        )
    else:
        analysis_collection = project_results_collection[0]

    results_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="ANALYSIS", collection=analysis_collection, 
        props = analysis_props, parents = analysis_parents
    )
    upload_datasets(results_object, analysis_support_files)
    
    display(utils.Javascript(data = "alert('Upload successful!')"))

def upload_datasets(method_object, support_files_widget):
    for filename in support_files_widget.value:
        file_info = support_files_widget.value[filename]
        utils.save_file(file_info['content'], filename)
        OPENBIS_SESSION.new_dataset(type = 'RAW_DATA', sample = method_object, files = [filename]).save()
        os.remove(filename)

# Create analysis

## Select project

In [ ]:
display(project_selector.dropdown_boxes)
project_selector.dropdown.observe(load_project_objects_metadata, names = 'value')

## Select simulations

In [ ]:
display(simulations_selector)

## Select measurements

In [ ]:
display(measurements_selector)

## Select code

In [ ]:
display(codes_selector)

## Properties

In [ ]:
display(analysis_widgets)

## Support files

In [ ]:
display(analysis_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_analysis_openbis)
quit_button.on_click(close_notebook)